### Import packages
Import first all important packages

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from yahoo_fin import stock_info as si
from yahoofinancials import YahooFinancials
import pandas
from csv import reader
import datetime

### Parse S&P 500 companies from wikipedia
Parse all 500 companies from wikipedia, since there are no API to do this.

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
r = requests.get(url)
r_html = r.text
soup = BeautifulSoup(r_html, 'html.parser')
components_table = soup.find_all(id="constituents")

### Save all companies in csv
Save all company tickers in a csv. This list of companies will be utilised later.

In [4]:
headers_html = soup.find_all("th")
df_columns = [item.text.rstrip("\n") for item in headers_html]
components_headers = df_columns[:9]
data_rows = components_table[0].find("tbody").find_all("tr")[1:]
rows = []
for row in range(len(data_rows)):
    stock = list(filter(None, data_rows[row].text.split("\n")))
    rows.append(stock)
S_P_500_stocks = pd.DataFrame(rows, columns=components_headers)
S_P_500_stocks.drop("SEC filings", inplace=True, axis=1)
S_P_500_stocks.to_csv(r"./data/SP500stocks.csv", index=False, encoding='utf-8')

## Let's start value calculation now

In [5]:
current_date = datetime.datetime.today().strftime('%d-%b-%Y')

### Get the recommendation for a ticker
This gets recommendation from yahoo finance.


In [6]:
def get_recommendation_rating(ticker):
    url = f'https://query2.finance.yahoo.com/v10/finance/quoteSummary/{ticker}?formatted=true&crumb=swg7qs5y9UP&lang=en-US&region=US&modules=upgradeDowngradeHistory,recommendationTrend,financialData,earningsHistory,earningsTrend,industryTrend&corsDomain=finance.yahoo.com'
    r = requests.get(url)
    if not r.ok:
        recommendation = 6
    try:
        result = r.json()['quoteSummary']['result'][0]
        recommendation = result['financialData']['recommendationMean']['fmt']
    except:
        recommendation = 6

    print("{} has an average recommendation of: ".format(ticker), recommendation)
    return recommendation